In [1]:
import numpy as np
import pandas as pd

In [2]:
transactions = pd.read_csv('hist_trx.csv')
marked = pd.read_csv('hits0712.csv')

In [3]:
transactions.drop('Unnamed: 0', axis=1, inplace=True)
marked.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
transactions.head()

,event_id,date_time,user_id,sub_channel,event_type,sub_type,atm_mcc,mcc_group,atm_merchant_name,amount
0,22c9f1ac686a43e18cdb798489193238,2018-12-06 09:33:21,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,280.0
1,62dac13fac68416d9bc340c51ddcb977,2018-12-06 09:36:08,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,376.9
2,9b666b8c9d4d4faea78e2b28a5468794,2018-12-06 12:21:02,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,143.0
3,d4d805fc3d5f4c91aa9b0389333b780c,2018-12-06 07:46:58,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,162.0
4,1d162351117a4418949dcf94111b9964,2018-12-06 12:21:27,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,56.0


In [5]:
marked.head()

,event_id,user_id,rec_user_id,date_time,resolution,sub_channel
0,0fcea7fec4ff479cac8cf37b4555c817,75301207,49913000,2018-12-07 07:56:49,G,ATMAPI
1,c29a4e64d27b435b9b55aa3e62ce54d4,1200695,12797310,2018-12-07 07:14:41,G,MOBILEAPI
2,3107b2b1afcb490ab0a31135eb4b386c,45657780,23814017,2018-12-07 14:02:30,G,MOBILEAPI
3,d9bc95d991144d53b950d8084fa846bf,21683486,9865131,2018-12-07 10:32:41,G,WEBAPI
4,9fc959da4cd144ea8bf5bae2dc041a5e,4531894,VSP15472255,2018-12-07 18:22:52,G,MOBILEAPI


In [ ]:
marked.resolution.unique()

In [6]:
fraud_mask = marked.resolution == 'F'
marked[fraud_mask].describe()

,event_id,user_id,rec_user_id,date_time,resolution,sub_channel
count,63,63,63,63,63,63
unique,63,55,46,63,1,2
top,4b832173a90e478abb7645125706a5d2,15965865,72300748,2018-12-07 15:45:57,F,MOBILEAPI
freq,1,4,4,1,63,59


In [8]:
marked.resolution.unique()

array(['G', 'F', 'S'], dtype=object)

In [9]:
marked[fraud_mask].groupby('sub_channel').count()

,event_id,user_id,rec_user_id,date_time,resolution
sub_channel,,,,,
MOBILEAPI,59,59,59,59,59
WEBAPI,4,4,4,4,4


Выборка не оч репрезентативна, но пока что все фроды были только через два канала. Через банкоматы пока нету.

In [10]:
marked[~fraud_mask].groupby('sub_channel').count()

,event_id,user_id,rec_user_id,date_time,resolution
sub_channel,,,,,
ATMAPI,1754,1754,1754,1754,1754
MOBILEAPI,6469,6469,6469,6469,6469
WEBAPI,730,730,730,730,730


In [11]:
transactions.set_index('user_id', drop=True, inplace=True)

In [12]:
transactions.head()

,event_id,date_time,sub_channel,event_type,sub_type,atm_mcc,mcc_group,atm_merchant_name,amount
user_id,,,,,,,,,
102050167,22c9f1ac686a43e18cdb798489193238,2018-12-06 09:33:21,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,280.0
102050167,62dac13fac68416d9bc340c51ddcb977,2018-12-06 09:36:08,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,376.9
102050167,9b666b8c9d4d4faea78e2b28a5468794,2018-12-06 12:21:02,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,143.0
102050167,d4d805fc3d5f4c91aa9b0389333b780c,2018-12-06 07:46:58,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,162.0
102050167,1d162351117a4418949dcf94111b9964,2018-12-06 12:21:27,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,56.0


In [24]:
froud_users_transactions = transactions.loc[[x for x in marked[fraud_mask]['rec_user_id'].values if x in transactions.index]]

In [18]:
marked[marked.rec_user_id == 'MBK29495696']

,event_id,user_id,rec_user_id,date_time,resolution,sub_channel
186,3e1a692d1e5b4874abf954307934ea95,85042196,MBK29495696,2018-12-07 16:19:00,F,MOBILEAPI
187,7ad6e07ad2074e8792ccc65c91458f9f,24945499,MBK29495696,2018-12-07 17:38:49,F,MOBILEAPI


In [25]:
froud_users_transactions

,event_id,date_time,sub_channel,event_type,sub_type,atm_mcc,mcc_group,atm_merchant_name,amount
user_id,,,,,,,,,
MBK29495696,e2a55a06e6264b1685c2f07a9c98419d,2018-12-06 15:06:47,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,P2P ROSBANK,4400.00
MBK29495696,13c697146cc1450b8131223311f85cd9,2018-12-06 13:28:14,ISSUER_ACQUIRER,WITHDRAW,ATM_WITHDRAW,6011.0,Z,ATM 60011906,4800.00
MBK29495696,11600c9a80ab47d78678a1546e5a4cc5,2018-12-06 13:08:01,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,P2P ROSBANK,4800.00
MBK29495696,47c2726575194b16ad1bdad61b802f69,2018-12-06 10:50:34,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,P2P ROSBANK,4400.00
MBK29495696,48f4323689a64102bf2b8b6ced000800,2018-12-06 16:37:37,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,OPEN.RU CARD2CARD,1500.00
MBK29495696,d587e236e5934f73ab10b905b50bea4a,2018-12-06 14:45:13,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,P2P ROSBANK,3400.00
MBK29495696,14f940fa075641d2bbf98d7a0f865ced,2018-12-06 18:11:26,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,P2P ROSBANK,2900.00
MBK29495696,baa599d587824c6ba7d389bb324f6924,2018-12-06 19:46:49,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,P2P ROSBANK,500.00
MBK29495696,ffbb8f9ab54a4b7ebd94c7962fbd6b08,2018-12-06 20:29:28,ISSUER,DEPOSIT,ATM_P2P_CREDIT,6536.0,R,P2P ROSBANK,2900.00


Транзакции в один рубль, видимо, для проверки работоспособности счета. Проверки баланса (даже несколько раз подряд)

In [31]:
froud_users_transactions.groupby('mcc_group').count()['event_id']/froud_users_transactions.shape[0]

mcc_group
A    0.018799
C    0.281706
F    0.007576
Q    0.000281
R    0.349046
T    0.001122
U    0.052469
X    0.000561
Z    0.287598
Name: event_id, dtype: float64

In [28]:
good_transactions = transactions.drop(froud_users_transactions.index)

In [32]:
good_transactions.groupby('mcc_group').count()['event_id']/good_transactions.shape[0]

mcc_group
A    0.002880
C    0.260973
F    0.044076
H    0.001389
J    0.000943
O    0.000127
Q    0.000257
R    0.341368
T    0.005143
U    0.207979
X    0.010233
Z    0.122602
Name: event_id, dtype: float64

Есть сильные отличия по категориям, однако, это может быть связанно просто с малым количеством данных( хотя тут уже их не так уж и мало)